# Predict With Our Amazon Comprehend Custom Classifier Model

<img src="img/comprehend.png" width="80%" align="left">

## Note that Amazon Comprehend is currently only supported in a subset of regions: 

* US East (N. Virginia), US East (Ohio), US West (Oregon)
* Canada (Central)
* Europe (London), Europe (Ireland), Europe (Frankfurt)
* Asia Pacific (Mumbai), Asia Pacific (Seoul), Asia Pacific (Tokyo), Asia Pacific (Singapore), Asia Pacific (Sydney)

You can check https://aws.amazon.com/about-aws/global-infrastructure/regional-product-services/ for details and updates. 

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

comprehend = boto3.Session().client(service_name="comprehend", region_name=region)

In [ ]:
%store -r comprehend_training_job_arn

In [ ]:
try:
    comprehend_training_job_arn
except NameError:
    print("***************************************************************************")
    print("[ERROR] PLEASE WAIT FOR THE PREVIOUS NOTEBOOK TO FINISH *******************")
    print("[ERROR] OR THIS NOTEBOOK WILL NOT RUN PROPERLY ****************************")
    print("***************************************************************************")

In [ ]:
print(comprehend_training_job_arn)

In [ ]:
%store -r comprehend_endpoint_arn

In [ ]:
try:
    comprehend_endpoint_arn
except NameError:
    print("***************************************************************************")
    print("[ERROR] PLEASE WAIT FOR THE PREVIOUS NOTEBOOK TO FINISH *******************")
    print("[ERROR] OR THIS NOTEBOOK WILL NOT RUN PROPERLY ****************************")
    print("***************************************************************************")

In [ ]:
print(comprehend_endpoint_arn)

# Deploy Endpoint

In [ ]:
describe_response = comprehend.describe_endpoint(EndpointArn=comprehend_endpoint_arn)
print(describe_response)

# Check Endpoint Status

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/comprehend/v2/home?region={}#classifier-details/{}/endpoints/{}/details">Comprehend Model Endpoint</a></b>'.format(
            region, comprehend_training_job_arn, comprehend_endpoint_arn
        )
    )
)

In [ ]:
import time

max_time = time.time() + 3 * 60 * 60  # 3 hours
while time.time() < max_time:
    describe_response = comprehend.describe_endpoint(EndpointArn=comprehend_endpoint_arn)
    status = describe_response["EndpointProperties"]["Status"]
    print("Endpoint: {}".format(status))

    if status == "IN_SERVICE" or status == "IN_ERROR":
        break

    time.sleep(5)

# [INFO] _Feel free to continue to the next workshop section while this notebook is running._

# Predict with Endpoint

In [ ]:
txt = """I loved it!  I will recommend this to everyone."""

response = comprehend.classify_document(Text=txt, EndpointArn=comprehend_endpoint_arn)

import json

print(json.dumps(response, indent=2, default=str))

In [ ]:
txt = """It's OK."""

response = comprehend.classify_document(Text=txt, EndpointArn=comprehend_endpoint_arn)

import json

print(json.dumps(response, indent=2, default=str))

In [ ]:
txt = """Really bad.  I hope they don't make this anymore."""

response = comprehend.classify_document(Text=txt, EndpointArn=comprehend_endpoint_arn)

import json

print(json.dumps(response, indent=2, default=str))

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}